#Майнор ИАД. Домашнее задание 3. YOLO.

В этом задании вы напишете и обучите свой собственный YOLO детектор. Нужно будет разобраться со статьей: понять какого формата должна быть обучающая пара (x, y), как перевести лосс из математической формулы в питоновский код - ну и конечно понять и реализовать саму архитектуру модели.

Выборка на котрой мы будем обучать модель состоит из разнообразных фотографий яблок, бананов и апельсинов. Данные скачиваем [отсюда](https://drive.google.com/file/d/1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3/view?usp=sharing).

Баллы за ДЗ распределены следующим образом: 
- Выборка для YoloV1 - 2 балла
- YOLO модель - 2 балла
- YOLO Loss - 3 балла
- Вспомогательные функции - 2 балла
- Обучение и расчет метрик - 2 балла

Для построения и обучения можно использовать как pytorch, так и pytorch-lightning.

Да-да, баллов в сумме получается 11

In [1]:
# Данная библиотека понадобится нам, чтобы обработать разметку
! pip install xmltodict pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Скачаем данные

In [ ]:
!wget --quiet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://drive.google.com/uc?export=download&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1d8GSfZoWbraWCSUhX78yl4CnMFYE-5n3" -O data.zip && rm -rf /tmp/cookies.txt
!unzip -q data.zip
!rm data.zip
!ls -l

Посмотрим как выглядит один из файлов разметки

In [ ]:
!cat data/train/apple_3.xml

## Релизуйте выборку для YoloV1 - 2 балла

In [2]:
import os
import cv2
import json
import glob
import tqdm
import xmltodict

from IPython.core.display import struct

from typing import List

import pandas as pd
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import pytorch_lightning as pl

import albumentations as A
import albumentations.pytorch

from PIL import Image

import matplotlib.pyplot as plt

from sklearn.metrics import auc
# Добавьте необходимые вам библиотеки, если их не окажется в списке выше

In [3]:
image_paths = glob.glob(os.path.join('data/train/','*.jpg'))

In [4]:
fileptr = open("/content/data/train/orange_76.xml","r")

#read xml content from the file
xml_content= fileptr.read()

#change xml format to ordered dict
my_ordered_dict=xmltodict.parse(xml_content)
my_ordered_dict

{'annotation': {'folder': 'train',
  'filename': 'orange_76.jpg',
  'path': 'C:\\tensorflow1\\models\\research\\object_detection\\images\\train\\orange_76.jpg',
  'source': {'database': 'Unknown'},
  'size': {'width': '1620', 'height': '1080', 'depth': '3'},
  'segmented': '0',
  'object': [{'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '494', 'ymin': '1', 'xmax': '1452', 'ymax': '741'}},
   {'name': 'orange',
    'pose': 'Unspecified',
    'truncated': '1',
    'difficult': '0',
    'bndbox': {'xmin': '822',
     'ymin': '643',
     'xmax': '1620',
     'ymax': '1066'}}]}}

In [5]:
len(my_ordered_dict['annotation']['object'])

2

In [6]:
dic_t = xmltodict.parse(open("/content/data/train/orange_76.xml","r").read())['annotation']['object']
dic_t[0]['bndbox']['xmax']

'1452'

In [7]:
im = np.array(Image.open('/content/data/train/mixed_8.jpg').convert("RGB"))
im.shape[0]

757

Так как в этом домашнем задании использовать аугментации для обучения __обязательно__ - советуем воспользоваться библиотекой albumentations.

Она  особенно удобна, поскольку умеет сама вычислять новые координаты bounding box'ов после трансформаций картинки. Для знакомства с этим механизмом советуем следующий [гайд](https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/). 

Вы все еще можете избрать путь torchvision.transforms, вам потребуется знакомый нам метод `__getitem__`, однако вычислять новые координаты bounding box'ов после трансформаций вам придётся вручную

__Обратите внимание__ на то, что в статье коробки предсказаний параметризуются через: _(x_center, y_center, width, height)_ (причем эти значения _относительные_), а в наших файлах - это _(x_min, y_min, x_max, y_max)_

Также, помните что модель должна предсказывать как прямоугольник с обьектом, так и вероятности каждого класса!

In [8]:
from sklearn import preprocessing

In [9]:
class2tag = {"apple": 1, "orange": 2, "banana": 3}

class FruitDataset(Dataset):
    def __init__(self, data_dir, transforms=None):
        self.image_paths = glob.glob( os.path.join(data_dir,'*.jpg'))
        self.enc = preprocessing.LabelEncoder()
        self.box_paths = [x+".xml" for x in [os.path.splitext(x)[0] for x in self.image_paths]]
        assert len(self.image_paths) == len(self.box_paths)
        self.transforms = transforms
        for idx in range(len(self.box_paths)):
            image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
            _,classes =  self.__get_boxes_from_xml(self.box_paths[idx],image)
            if idx == 0:
              all_classes = classes
            classes = np.array(classes)
            all_classes = np.concatenate((all_classes,classes))
        all_classes = np.unique(all_classes)
        self.enc.fit(all_classes)


    # Координаты прямоугольников советуем вернуть именно в формате (x_center, y_center, width, height)
    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        boxes, class_labels = self.__get_boxes_from_xml(self.box_paths[idx],image)
        if self.transforms is not None:
          augs = self.transforms(image=image, bboxes = boxes, class_labels=class_labels)
          image = augs['image']
          boxes = augs['bboxes']
          class_labels = augs['class_labels']
        boxes, class_labels = torch.as_tensor(boxes, dtype=torch.float64),  torch.as_tensor(self.enc.transform(class_labels),dtype=torch.int8)
        return image.unsqueeze(dim=0), (boxes, class_labels)

    def __len__(self):
        return len(self.image_paths)

    def __get_boxes_from_xml(self, xml_filename: str,image):
      """
      Метод, который считает и распарсит (с помощью xmltodict) переданный xml
      файл и вернет координаты прямоугольников обьектов на соответсвующей фотографии
      и название класса обьекта в каждом прямоугольнике

      Обратите внимание, что обьектов может быть как несколько, так и один единственный
      """
      boxes = []
      class_labels = []
      dic = xmltodict.parse(open(xml_filename,"r").read())['annotation']
      im_w = int(dic['size']['width'])
      if im_w == 0:
        im_w = image.shape[1]
      im_h = int(dic['size']['height'])
      if im_h == 0:
        im_h = image.shape[0]
      obj = dic['object']
      if isinstance(obj, dict):
        obj = [obj]
      for item in obj:
        boxes.append(self.__convert_to_yolo_box_params(
            [int(x) for x in [item['bndbox']['xmin'], item['bndbox']['ymin'], item['bndbox']['xmax'], item['bndbox']['ymax']]],
            im_w,im_h)
        ) 
        class_labels.append(item['name'])      
      return boxes,class_labels

    def __convert_to_yolo_box_params(self, box_coordinates: List[int], im_w, im_h):
      """
      Перейти от [xmin, ymin, xmax, ymax] к [x_center, y_center, width, height].
      
      Обратите внимание, что параметры [x_center, y_center, width, height] - это
      относительные значение в отрезке [0, 1]

      :param: box_coordinates - координаты коробки в формате [xmin, ymin, xmax, ymax]
      :param: im_w - ширина исходного изображения
      :param: im_h - высота исходного изображения

      :return: координаты коробки в формате [x_center, y_center, width, height]
      """
      ans = [] 
      ans.append((box_coordinates[0] + box_coordinates[2]) / 2 / im_w)# x_center  
      ans.append((box_coordinates[1] + box_coordinates[3]) / 2 / im_h)  # y_center
      ans.append((box_coordinates[2] - box_coordinates[0]) / im_w)# width  
      ans.append((box_coordinates[3] - box_coordinates[1]) / im_h)  # height 
      return ans

In [10]:
transform = A.Compose([A.ToFloat (max_value=None, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()
                       ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
train_dataset = FruitDataset(transforms=transform,
    data_dir="./data/train")
train_dataloader = DataLoader(dataset=train_dataset, batch_size=1)
test_dataset = FruitDataset(transforms=transform,
    data_dir="./data/test")
test_dataloader = DataLoader(dataset=test_dataset, batch_size=1)

/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [11]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        print(data)
        channels_sum += torch.mean(data[0], dim=[0,2,3])
        channels_squared_sum += torch.mean(data[0]**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [12]:
mean, std = get_mean_and_std(train_dataloader)
mean_tr,std_tr = mean.tolist(),std.tolist()
mean, std = get_mean_and_std(test_dataloader)
mean_test,std_test = mean.tolist(),std.tolist()

Выходные данные были обрезаны до нескольких последних строк (5000).
           [0.3098, 0.3020, 0.2902,  ..., 0.5804, 0.5804, 0.5804]],

          [[0.0078, 0.0078, 0.0078,  ..., 0.6667, 0.6667, 0.6667],
           [0.0118, 0.0157, 0.0118,  ..., 0.6627, 0.6588, 0.6549],
           [0.0157, 0.0118, 0.0118,  ..., 0.6549, 0.6431, 0.6314],
           ...,
           [0.1020, 0.1059, 0.1020,  ..., 0.9137, 0.9137, 0.9137],
           [0.1020, 0.0980, 0.1020,  ..., 0.9137, 0.9137, 0.9137],
           [0.1020, 0.0941, 0.0941,  ..., 0.9137, 0.9137, 0.9137]]]]])
tensor([[[[[0.7255, 0.7451, 0.7647,  ..., 0.2392, 0.2510, 0.2627],
           [0.7333, 0.7608, 0.7882,  ..., 0.2314, 0.2196, 0.2039],
           [0.7333, 0.7608, 0.7804,  ..., 0.2431, 0.2275, 0.2157],
           ...,
           [0.0745, 0.0667, 0.0784,  ..., 0.3333, 0.3255, 0.3059],
           [0.0824, 0.0627, 0.0745,  ..., 0.3333, 0.3412, 0.3333],
           [0.1255, 0.1020, 0.1137,  ..., 0.3412, 0.3765, 0.3843]],

          [[0.4588, 0

In [13]:
mean_tr

[0.8134564757347107, 0.6914445757865906, 0.5386322736740112]

In [14]:
std_tr

[0.2758118212223053, 0.3231789469718933, 0.40659835934638977]

In [15]:
mean_test

[0.8366212248802185, 0.7004530429840088, 0.5619865655899048]

In [16]:
std_test

[0.25586509704589844, 0.3319559693336487, 0.40674322843551636]

In [17]:
WIDTH, HEIGHT = 448,448

train_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_tr, std=std_tr, max_pixel_value=255.0, always_apply=False, p=1.0),A.Resize(WIDTH, HEIGHT),
    A.pytorch.transforms.ToTensorV2(), ], 
                             bbox_params=A.BboxParams(format='yolo',
                                                      label_fields=['class_labels']))
test_transform = A.Compose([A.HorizontalFlip(p=0.5),
    A.ColorJitter (brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, always_apply=False, p=0.5),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.Normalize (mean=mean_test, std=std_test, max_pixel_value=255.0, always_apply=False, p=1.0),
    A.pytorch.transforms.ToTensorV2()],
                            bbox_params=A.BboxParams(format='yolo',
                                                     label_fields=['class_labels']))

In [18]:
train_dataset = FruitDataset(
    transforms=train_transform,
    data_dir="./data/train"
    )

val_dataset = FruitDataset(
    transforms=test_transform, 
    data_dir="./data/test"
    )

# Немного проверок, чтобы убедиться в правильности направления решения
assert isinstance(train_dataset[2], tuple)
assert len(train_dataset[2]) == 2
assert isinstance(train_dataset[2][0], torch.Tensor)
assert isinstance(train_dataset[2][1], tuple)
assert len(train_dataset[2][1]) == 2
print("Тесты успешно пройдены")

Тесты успешно пройдены


In [19]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [20]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 16,
    num_workers = 4,
    shuffle=True, 
    collate_fn=collate_fn
    )

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=16,
    num_workers = 4, 
    shuffle=False,
    collate_fn=collate_fn
    )

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Теперь определим функцию для рассчета Intersection Over Union по 4 углам двух прямоугольников

In [21]:
def intersection_over_union(predicted_bbox, gt_bbox) -> float:
    """
    Intersection Over Union для двух прямоугольников

    :param: predicted_bbox - [x_min, y_min, x_max, y_max]
    :param: gt_bbox - [x_min, y_min, x_max, y_max]
    
    :return: Intersection Over Union
    """

    intersection_bbox = np.array(
        [
            max(predicted_bbox[0], gt_bbox[0]),
            max(predicted_bbox[1], gt_bbox[1]),
            min(predicted_bbox[2], gt_bbox[2]),
            min(predicted_bbox[3], gt_bbox[3]),
        ]
    )

    intersection_area = max(intersection_bbox[2] - intersection_bbox[0], 0) * max(
        intersection_bbox[3] - intersection_bbox[1], 0
    )
    area_dt = (predicted_bbox[2] - predicted_bbox[0]) * (predicted_bbox[3] - predicted_bbox[1])
    area_gt = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])

    union_area = area_dt + area_gt - intersection_area

    iou = intersection_area / union_area
    return iou

In [22]:
x = next(iter(train_dataloader))
print(x)

((tensor([[[[-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
          [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
          [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
          ...,
          [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
          [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
          [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493]],

         [[-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
          [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
          [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
          ...,
          [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
          [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
          [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395]],

         [[-1.3247, -1.3247, -1.3247,  ..., -1.3247, -1.3247, -1.3247],
          [-1.3247, -1.3247,

In [23]:
x[0]

(tensor([[[[-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
           [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
           [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
           ...,
           [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
           [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493],
           [-2.9493, -2.9493, -2.9493,  ..., -2.9493, -2.9493, -2.9493]],
 
          [[-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
           [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
           [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
           ...,
           [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
           [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395],
           [-2.1395, -2.1395, -2.1395,  ..., -2.1395, -2.1395, -2.1395]],
 
          [[-1.3247, -1.3247, -1.3247,  ..., -1.3247, -1.3247, -1.3247],
           [-

Теперь начинается основная часть домашнего задания: обучите модель YOLO для object detection на __обучающем__ датасете. 

 - Создайте модель и функцию ошибки YoloV1 прочитав [оригинальную статью](https://paperswithcode.com/paper/you-only-look-once-unified-real-time-object)
 - Напишите функцию обучения модели
 - Используйте аугментации

## Реализуйте Модель - 2 балла

Копировать точное количество слоев и параметры сверток необязательно. Главное - чтобы модель работала по принципу, описанному в статье и делала предсказание в представленном формате.


В качестве подсказки напомним, что выходом модели __для каждого обьекта__ должен быть тензор размера
__S * S * (B * 5 + С)__, где все параметры имеют такое же значение, как и в статье: 

- S - количество ячеек на которое разбивается изображение по вертикали/горизонтали
- В - количество предсказываемых прямоугольников в каждой ячейке
- 5 - количество параметров для определения каждого прямоугольника (x_center, y_center, width, height, confidence)
- С - количество классов (apple, banana, orange)

Таким образом, мы для каждого окна размера __S x S__ предсказываем __В__ коробо и один класс

In [24]:
params = [ #[channels, Maxpool, kernel, padding, stride]
    [64,True,7,3,2],
    [192,True,3,1,1],
    [128,False,1,0,1],
    [256,False,3,1,1],
    [256,False,1,0,1],
    [512,True,3,1,1],
    (4,
    [[256,False,1,0,1],
    [512,False,3,1,1]]),
    [512,False,1,0,1],
    [1024,True,3,1,1],
    (2,
    [[512,False,1,0,1],
    [1024,False,3,1,1]]),
    [1024,False,3,1,1],
    [1024,False,3,1,2],
    (2,[[1024,False,3,1,1]])
]

In [25]:
class CNNBlock(nn.Module):  # можно поменять на Lightning
    def __init__(self, in_channels, out_channels, is_max_pool:bool=False, **kwargs):
        
        super().__init__()

        self.conv = nn.Conv2d(in_channels, out_channels, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)  # в статье еще не знали про батчнорм, но мы то из будущего ...
        self.leakyrelu = nn.LeakyReLU(0.1)

        self.is_maxpool = is_max_pool  # не после каждой свертки нужно делать maxpool
        self.maxpool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.leakyrelu(self.batchnorm(self.conv(x)))

        if self.is_maxpool:
            x = self.maxpool(x)

        return x
    
class YOLO(nn.Module):
    def __init__(self, S=7, B=2, C=3,in_channels:int = 3, architecture:list=params,**kwargs):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super(YOLO, self).__init__()

        self.S = S
        self.B = B
        self.C = C
        self.in_channels=in_channels
        self.arch = architecture
        self.net = self._create_conv_layers()
        self.fc = self._create_fcs(**kwargs)

    def forward(self, x):
        for block in self.net:
          x= block(x)
        return self.fc(x)


    def _create_conv_layers(self):
      in_channels = self.in_channels
      conv = nn.ModuleList()
      for block in self.arch:
        if isinstance(block, list):
          conv.append(CNNBlock(in_channels,*block[0:2],kernel_size=block[2],padding=block[3],stride=block[4]))
          in_channels = block[0]
        elif isinstance(block, tuple):
          for _ in range(block[0]):
            for small_block in block[1]:
              conv.append(CNNBlock(in_channels,*small_block[0:2],kernel_size=small_block[2],padding=small_block[3],stride=small_block[4]))
              in_channels=small_block[0]
        else:
          raise TypeError("architecture can include only types: 'list' or 'tuple'")
      return conv


    def _create_fcs(self,p = 0.1):
      return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * self.S * self.S, 4096),
            nn.Dropout(p = p),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, self.S * self.S * (self.C + self.B * 5)),
        )

temp_model = YOLO()
expected_output_shape = temp_model.S * temp_model.S * (5 * temp_model.B + temp_model.C)

assert temp_model(torch.cat((train_dataset[0][0],train_dataset[1][0]))).shape[1] == expected_output_shape
temp_model(train_dataset[0][0]).size()

torch.Size([1, 637])

In [26]:
temp_model = YOLO()
g = temp_model(torch.cat(x[0]))

In [27]:
print(g.size())
g.shape[1]

torch.Size([16, 637])


637

## Реализуйте YoloLoss - 3 балла

In [29]:
from itertools import product

In [30]:
def target_to_matrix(boxes, labels,S=7, B=2, C=3):
  label_matrix = torch.zeros((C + 5,S, S ))
  for box,i in enumerate(boxes):
    x,y,w,h = box.tolist()
    class_label = labels[i]
    class_label = int(class_label)

    i, j = int(S * y), int(S * x)
    x_cell, y_cell = S * x - j, S * y - i
    width_cell, height_cell = (w * S, h * S)

    if label_matrix[C,i, j] == 0:
        label_matrix[C,i, j] = 1
        box_coordinates = torch.tensor(
            [x_cell, y_cell, width_cell, height_cell]
        )

        label_matrix[C+1:,i, j ] = box_coordinates

        # Set one hot encoding for class_label
        label_matrix[class_label,i, j ] = 1

        return label_matrix

In [31]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=3):
        """
        :param: S * S - количество ячеек на которые разбивается изображение
        :param: B - количество предсказанных прямоугольников в каждой ячейке
        :param: C - количество классов
        """
        
        super().__init__()
        self.mse = nn.MSELoss(reduction="sum")

        self.S = S
        self.B = B
        self.C = C

        self.lambda_noobj = 0.5
        self.lambda_coord = 5

    def forward(self, predictions, target):
        self.pred = predictions.reshape(self.C+5*self.B,self.S,self.S)
        loss = 0
        iou = torch.empty((self.B,self.S,self.S))
        box_pred = torch.empty((4,self.S,self.S))
        ######################################
        ###           Cord Loss            ###
        ######################################
        for x,y,n in product(range(self.S),range(self.S),range(self.B)):
          iou[n,x,y] = intersection_over_union(self.pred[self.C+5*n+1:self.C+5*n+5,x,y],self.target[self.C+1:self.C+5,...])
        best_boxes = torch.argmax(iou,dim=0)
        I_obj = target[self.C,...]
        for x,y in product(range(self.S),range(self.S)):
          box_pred[:,x,y]=self.pred[self.C+5*int(best_boxes[x,y])+1:self.C+5*int(best_boxes[x,y])+5,x, y]
        box_pred = I_obj*box_pred
        box_targ = I_obj * target[self.C+1:self.C+5,...]
        box_pred[2:4,...],box_targ[2:4,...] = torch.sqrt(torch.abs(box_pred[2:4,...])),torch.sqrt(torch.abs(box_targ[2:4,...])) 
        loss +=self.lambda_coord*self.mse(torch.flatten(box_pred),torch.flatten(box_targ))
        ######################################
        ###            Obj Loss            ###
        ######################################
        obj_pred = torch.empty((self.S,self.S))
        for x,y in product(range(self.S),range(self.S)):
          obj_pred[x,y]=self.pred[self.C+5*int(best_boxes[x,y]),x, y]
        obj_pred = I_obj*obj_pred
        obj_targ = I_obj* target[self.C,...]
        loss += self.mse(torch.flatten(obj_pred),torch.flatten(obj_targ))
        ######################################
        ###           No Obj Loss          ###
        ######################################
        I_noobj = 1-I_obj
        noobj_targ = I_noobj*target[self.C,...]
        for i in range(self.B):
          noobj_pred = I_noobj*self.pred[self.C+5*i,x, y]
          loss += self.lambda_noobj*self.mse(torch.flatten(noobj_pred),torch.flatten(noobj_targ))
        ######################################
        ###           Class Loss           ###
        ######################################
        loss += self.mse(
            torch.flatten(I_obj * self.pred[:self.C,...]),
            torch.flatten(I_obj * target[:self.C,... ] ),
        )
        return loss

## Реализуйте дополнительные функции из статьи - 2 балла

In [33]:
def get_boxes_from_pred(pred,S=7, B=2, C=3):
  batch = pred.shape[0]
  pred = pred.reshape(batch,C+5*B,S,S)
  best_boxes = torch.empty((batch,B,S,S))
  cell_pred = torch.empty((batch,S,S,1+5))
  for x in product(range(B)):
    best_boxes[:,x,...] = pred[:,C+5*x,...]
  best_boxes = torch.argmax(best_boxes,dim=1)
  for x,y,n in product(range(batch),range(S),range(S)):
    cell_pred[x,y,n,0] = torch.argmax(pred[x,:C,y,n])
    cell_pred[x,y,n,1:] = pred[x,C+5*int(best_boxes[x,y,n]):C+5*int(best_boxes[x,y,n])+5,y,n]
  cell_pred = cell_pred.flatten(1,2)
  return cell_pred

In [44]:
def get_target(target):
  target = np.array([list(i) for i in target])
  print(target)
  targ_arr = []
  boxes,labels = target[:,1],target[:,0]
  boxes = boxes.tolist()
  boxes = labels.tolist()
  print(boxes)
  for i in range(len(boxes)):
    vec = []
    for label, box in product(labels,boxes):
      vec.append([i]+label.tolist()+[1]+box.tolist())
    targ_arr.append(vec)
  return np.array(targ_arr)

In [45]:
from collections import Counter 

In [46]:
def non_max_suppression(bboxes, iou_threshold, threshold,S=7):
    """
    bboxes : tensor shape[S*S,C+5]
    bboxes[i]: [class label,probability,x,y,w,h]
    """
    bboxes = bboxes.tolist()
    assert len(bboxes[0]) == 1+5
    assert len(bboxes) == S*S
    bboxes = [box for box in bboxes if box[1] > threshold]
    bboxes = sorted(bboxes, key=lambda x: x[1], reverse=True)
    bboxes_after_nms = []

    while bboxes:
        chosen_box = bboxes.pop(0)

        bboxes = [
            box
            for box in bboxes
            if box[0] != chosen_box[0]
            or intersection_over_union(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
            )
            < iou_threshold
        ]

        bboxes_after_nms.append(chosen_box)

    return bboxes_after_nms
    

def mean_average_precision(pred_boxes, true_boxes, iou_threshold=0.5,threshold_step=.05,threshold_end = 0.5,C=3):
    vec_precisions = []

    for class_label in range(C):
      pred = []
      test = []
      for vec in pred_boxes:
            if vec[1] == class_label:
                pred.append(vec)

      for vec in true_boxes:
            if vec[1] == class_label:
                test.append(vec)
      n = len(test)
      if n == 0:
        continue 
     
      pred.sort(key=lambda x: x[2], reverse=True)
      for threshold in np.arange(iou_threshold,threshold_end+threshold_step/10,threshold_step):
        true_p = torch.zeros((len(pred)))
        false_p = torch.zeros((len(pred)))
        boxes_in_image = Counter([x[0] for x in test])
        for i,x in boxes_in_image:
          boxes_in_image[i] = torch.zeros(x)
        for i_d, vec_pred in enumerate(pred):
          test_image = [x for x in test if x[0]==vec_pred[0]]
          best_iou = 0
          for i_b, vec_test in enumerate(test_image):
            iou = intersection_over_union(vec_pred[2:], vec_test[2:])
            if iou > best_iou:
              best_iou = iou
              idx_best = i_b
          if best_iou > threshold:
            if boxes_in_image[vec_pred[0]][idx_best] == 0:
              true_p[i_d] = 1
              boxes_in_image[vec_pred[0]][idx_best] = 1
            else: 
              false_p[i_d] = 1
          else:
            false_p[i_d]=1
        
        true_p = torch.cumsum(true_p, dim=0)
        false_p = torch.cumsum(false_p, dim=0)
        recalls = true_p/ (len(test) + 1e-6)
        precisions = torch.divide(true_p, (true_p + false_p + 1e-6))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        vec_precisions.append(torch.trapz(precisions, recalls))  
    return sum(vec_precisions)/len(vec_precisions)         

def get_bound_boxes(loader, model, iou_threshold=.5, threshold=.4):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    all_pred_boxes = []
    all_true_boxes = []
    model.eval()
    train_idx = 0
    for x,label in loader:
      x = torch.cat(x).to(device)
      with torch.no_grad():
        predictions = model(x)
      batch_size = x.shape[0]
      true_bboxes = get_target(label)
      bboxes = get_boxes_from_pred(predictions)
      for idx in range(batch_size):
          nms_boxes = non_max_suppression(
              bboxes[idx],
              iou_threshold=iou_threshold,
              threshold=threshold,
          )
          for nms_box in nms_boxes:
              all_pred_boxes.append([train_idx] + nms_box)

          for box in true_bboxes[idx]:
              if box[1] > threshold:
                  all_true_boxes.append([train_idx] + box)

          train_idx += 1
    return all_pred_boxes, all_true_boxes

In [37]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [38]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size= 2,
    num_workers = 4,
    shuffle=True, 
    collate_fn=collate_fn
    )

val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=2,
    num_workers = 4, 
    shuffle=False,
    collate_fn=collate_fn
    )

In [47]:
get_bound_boxes(train_dataloader,YOLO())

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
<ipython-input-44-441d9e74e87d>:2: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interf

[[tensor([[0.4840, 0.5290, 0.8120, 0.8380]], dtype=torch.float64)
  tensor([0], dtype=torch.int8)]
 [tensor([[0.6224, 0.5814, 0.5056, 0.8371],
          [0.4288, 0.3900, 0.5952, 0.5000],
          [0.4544, 0.4986, 0.5856, 0.7400],
          [0.4216, 0.4371, 0.5488, 0.5829]], dtype=torch.float64)
  tensor([1, 1, 1, 1], dtype=torch.int8)]]
[tensor([[0.4840, 0.5290, 0.8120, 0.8380]], dtype=torch.float64), tensor([[0.6224, 0.5814, 0.5056, 0.8371],
        [0.4288, 0.3900, 0.5952, 0.5000],
        [0.4544, 0.4986, 0.5856, 0.7400],
        [0.4216, 0.4371, 0.5488, 0.5829]], dtype=torch.float64)]


TypeError: ignored

In [155]:
for x,y in train_dataloader:
  break

In [156]:
torch.cat(x).size()

torch.Size([2, 3, 448, 448])

In [159]:
y[0][0].tolist()

[[0.7153225806451613,
  0.5435835351089588,
  0.22741935483870968,
  0.5835351089588379],
 [0.5483870967741935,
  0.5411622276029056,
  0.13548387096774195,
  0.6319612590799033],
 [0.4120967741935484,
  0.5556900726392252,
  0.11774193548387096,
  0.5786924939467313],
 [0.2717741935483871,
  0.5702179176755447,
  0.15322580645161288,
  0.5738498789346247]]

In [100]:
labels = np.array([list(i) for i in label])

<ipython-input-100-91d4b15cbab2>:1: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  labels = np.array([list(i) for i in label])
<ipython-input-100-91d4b15cbab2>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  labels = np.array([list(i) for i in label])


In [102]:
labels

array([[tensor([[0.2646, 0.3599, 0.3848, 0.3721],
                [0.7329, 0.3486, 0.3916, 0.3770],
                [0.4946, 0.7666, 0.6650, 0.3535]], dtype=torch.float64),
        tensor([2, 0, 1], dtype=torch.int8)],
       [tensor([[0.5580, 0.4981, 0.8270, 0.6734],
                [0.3828, 0.5991, 0.7565, 0.8018],
                [0.6013, 0.3592, 0.7965, 0.4347]], dtype=torch.float64),
        tensor([1, 1, 1], dtype=torch.int8)]], dtype=object)

## Обучите модель и посчитайте метрики для задачи детекции - 2 балла 

Несмотря на то, что в этом блоке ничего сильно нового для вас не ожидается и за него формально дается лишь два балла - провести обучение очень важно для понимания того, насколько правильно реализована ваша модель и лосс.

В процессе обучения будет видно все ли размерности совпадают, падает ли лосс и растут ли метрики целевой задачи, поэтому на практике этот пункт гораздо оказывается гораздо важнее.

In [ ]:
class YOLOLearner(pl.LightningModule):
    def __init__(self) -> None:
        super().__init__()

        self.model = YOLO()
        self.loss = YoloLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def forward(self, x) -> torch.Tensor:
        return self.model(x)

    def configure_optimizers(self):
        return self.optimizer

    def training_step(self, train_batch, batch_idx) -> torch.Tensor:
        ## YOUR CODE
        pass

    def validation_step(self, val_batch, batch_idx) -> None:
        ## YOUR CODE
        pass

In [ ]:
model = # YOUR CODE
n_epochs = # YOUR CODE

yolo_learner = YOLOLearner(...)  ## YOUR CODE

device = "gpu" if torch.cuda.is_available() else "cpu"
trainer = pl.Trainer(accelerator=device, max_epochs=n_epochs)

trainer.fit(yolo_learner, train_dataloader, val_dataloader)

## Посчитайте метрики задачи детекции на валидационной выборке

Попробуйте понять насколько хороши ваши показатели. Если числа кажутся подозрительно низкими - возможно вам стоит перепроверить ваше решение. 

In [ ]:
## YOUR CODE

## Визуализируйте предсказанные bounding box'ы для любых пяти картинок из __валидационного__ датасета.

In [ ]:
image, targets = next(iter(val_dataset))
preds = ## YOUR CODE

In [ ]:
from PIL import ImageDraw

image = torchvision.transform.ToPILImage()(image)
draw = ImageDraw.Draw(image)

for box in targets[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])])

for box in preds[0]:
    ## YOUR CODE
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline='red')
image